In [14]:
pip install yfinance
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.


In [39]:
import yfinance as yf
import pandas as pd
import numpy as np
import logging
from pandas_datareader import data as pdr

In [15]:
import yfinance as yf
import pandas as pd

In [21]:
def fetch_data(tickers, start_date, end_date):
    """
    Fetches historical stock data from Yahoo Finance and adjusts the DataFrame format for ease of use.

    :param tickers: List of stock ticker symbols as a string separated by spaces if multiple, e.g., 'AAPL MSFT'.
    :param start_date: Start date for the data in 'YYYY-MM-DD' format.
    :param end_date: End date for the data in 'YYYY-MM-DD' format.
    :return: DataFrame containing historical stock data, indexed by date with multi-level columns if multiple tickers.
    """
    data = yf.download(tickers, start=start_date, end=end_date)
    if len(tickers.split()) > 1:
        # Ensuring the data is returned with a multi-level column index when multiple tickers are queried
        data = data['Adj Close']  # You can change this if you need other data like 'Open', 'Close', etc.
    else:
        # Adjusting for a single ticker query - flattening the DataFrame
        data.columns = data.columns.droplevel(0)
    return data


In [22]:
# Example usage:
tickers = 'AAPL MSFT GOOGL'  # String of ticker symbols separated by spaces
start_date = '2018-01-01'
end_date = '2023-01-01'

# Fetching data
stock_data = fetch_data(tickers, start_date, end_date)

# Display the first few rows of the fetched data
print(stock_data.head())


[*********************100%%**********************]  3 of 3 completed

Ticker           AAPL      GOOGL       MSFT
Date                                       
2018-01-02  40.670986  53.660500  80.080917
2018-01-03  40.663879  54.576000  80.453629
2018-01-04  40.852776  54.787998  81.161720
2018-01-05  41.317898  55.514500  82.167976
2018-01-08  41.164429  55.710499  82.251816


In [31]:
def fetch_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data
test_data = fetch_data('AAPL', '2023-01-01', '2023-05-01')
print(test_data.head())

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-01-03  130.279999  130.899994  124.169998  125.070000  124.216301   
2023-01-04  126.889999  128.660004  125.080002  126.360001  125.497498   
2023-01-05  127.129997  127.769997  124.760002  125.019997  124.166641   
2023-01-06  126.010002  130.289993  124.889999  129.619995  128.735245   
2023-01-09  130.470001  133.410004  129.889999  130.149994  129.261627   

               Volume  
Date                   
2023-01-03  112117500  
2023-01-04   89113600  
2023-01-05   80962700  
2023-01-06   87754700  
2023-01-09   70790800  


In [32]:
def fetch_data(ticker):
    data = yf.download(ticker, start="2023-01-01", end="2023-12-31")
    data['MA50'] = data['Adj Close'].rolling(window=50).mean()
    data['MA200'] = data['Adj Close'].rolling(window=200).mean()
    return data

def filter_stocks(ticker):
    data = fetch_data(ticker)
    latest_data = data.iloc[-1]  # Get the latest data point
    if latest_data['Adj Close'] > latest_data['MA50'] > latest_data['MA200']:
        return True
    else:
        return False

# Example usage:
tickers = ['AAPL', 'MSFT', 'GOOGL']
filtered_stocks = [ticker for ticker in tickers if filter_stocks(ticker)]
print("Stocks in uptrend:", filtered_stocks)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Stocks in uptrend: ['AAPL', 'MSFT', 'GOOGL']


In [33]:
import yfinance as yf
import pandas as pd
import numpy as np
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_tickers():
    """
    Fetches a complete list of tickers. This function assumes a placeholder implementation.
    For actual use, replace it with a function that fetches live ticker data from a reliable source.
    """
    return ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']  # Placeholder list

def fetch_data(ticker):
    """
    Fetch historical data for the given ticker and compute moving averages.
    """
    try:
        data = yf.download(ticker, period="1y")
        data['MA50'] = data['Adj Close'].rolling(window=50).mean()
        data['MA200'] = data['Adj Close'].rolling(window=200).mean()
        return data
    except Exception as e:
        logging.error(f"Failed to download data for {ticker}: {str(e)}")
        return None

def is_in_uptrend(data):
    """
    Determine if the stock is in uptrend based on the last available data.
    """
    if data is not None and not data.empty:
        last_close = data.iloc[-1]
        return last_close['Adj Close'] > last_close['MA50'] > last_close['MA200'] and last_close['MA50'] > last_close['MA200']
    return False

def process_batch(tickers):
    """
    Process a batch of tickers and return those in an uptrend.
    """
    uptrend_stocks = []
    for ticker in tickers:
        data = fetch_data(ticker)
        if is_in_uptrend(data):
            uptrend_stocks.append(ticker)
            logging.info(f"{ticker} is in an uptrend.")
        else:
            logging.info(f"{ticker} is not in an uptrend.")
    return uptrend_stocks

def main():
    tickers = fetch_tickers()
    batch_size = 5  # Number of tickers to process at once
    uptrend_stocks = []
    
    for i in range(0, len(tickers), batch_size):
        batch_tickers = tickers[i:i+batch_size]
        uptrend_stocks.extend(process_batch(batch_tickers))
    
    logging.info(f"Total stocks in uptrend: {len(uptrend_stocks)}")
    print("Stocks in uptrend:", uptrend_stocks)

if __name__ == "__main__":
    main()


[*********************100%%**********************]  1 of 1 completed
2024-05-06 16:03:15,308 - INFO - AAPL is not in an uptrend.
[*********************100%%**********************]  1 of 1 completed
2024-05-06 16:03:15,450 - INFO - MSFT is not in an uptrend.
[*********************100%%**********************]  1 of 1 completed
2024-05-06 16:03:15,581 - INFO - GOOGL is in an uptrend.
[*********************100%%**********************]  1 of 1 completed
2024-05-06 16:03:15,820 - INFO - AMZN is in an uptrend.
[*********************100%%**********************]  1 of 1 completed
2024-05-06 16:03:16,049 - INFO - TSLA is not in an uptrend.
2024-05-06 16:03:16,049 - INFO - Total stocks in uptrend: 2


Stocks in uptrend: ['GOOGL', 'AMZN']


In [46]:

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_tickers():
    """
    Fetches a complete list of NASDAQ tickers using pandas_datareader.
    """
    try:
        nasdaq_tickers = pdr.get_nasdaq_symbols()
        return nasdaq_tickers.index.tolist()
    except Exception as e:
        logging.error(f"Failed to fetch NASDAQ tickers: {str(e)}")
        return []


In [47]:
def fetch_data(ticker):
    """
    Fetch historical data for the given ticker and compute moving averages.
    """
    try:
        data = yf.download(ticker, period="1y")
        data['MA50'] = data['Adj Close'].rolling(window=50).mean()
        data['MA200'] = data['Adj Close'].rolling(window=200).mean()
        return data
    except Exception as e:
        logging.error(f"Failed to download data for {ticker}: {str(e)}")
        return None

In [48]:
def is_in_uptrend(data):
    """
    Determine if the stock is in uptrend based on the last available data.
    """
    if data is not None and not data.empty:
        last_close = data.iloc[-1]
        return last_close['Adj Close'] > last_close['MA50'] > last_close['MA200'] and last_close['MA50'] > last_close['MA200']
    return False

In [ ]:
def process_batch(tickers):
    """
    Process a batch of tickers and return a DataFrame of those in an uptrend with their last close, MA50, and MA200.
    """
    uptrend_stocks = []
    for ticker in tickers:
        data = fetch_data(ticker)
        if data is not None and is_in_uptrend(data):
            last_data = data.iloc[-1]
            uptrend_stocks.append({
                "Ticker": ticker,
                "Last Close": last_data['Adj Close'],
                "MA50": last_data['MA50'],
                "MA200": last_data['MA200']
            })
            logging.info(f"{ticker} is in an uptrend.")
        else:
            logging.info(f"{ticker} is not in an uptrend.")
    return pd.DataFrame(uptrend_stocks)


In [ ]:
def main():
    tickers = fetch_tickers()
    batch_size = 100  # Adjust based on your system capacity and rate limits
    results = []

    for i in range(0, len(tickers), batch_size):
        batch_tickers = tickers[i:i+batch_size]
        results.append(process_batch(batch_tickers))
    
    uptrend_stocks_df = pd.concat(results, ignore_index=True)
    logging.info(f"Total stocks in uptrend: {uptrend_stocks_df.shape[0]}")
    print(uptrend_stocks_df)

if __name__ == "__main__":
    main()
